<a href="https://colab.research.google.com/github/Liang130520/Tibame-Machine-Learning/blob/main/class1103_machine_learning_Kaggle_Titanic_9_One_Hot_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
" Kaggle "
from urllib.request import urlretrieve

url = "https://github.com/Elwing-Chou/tibaml1017/raw/main/titanic/train.csv"
urlretrieve(url, "train.csv")

url = "https://github.com/Elwing-Chou/tibaml1017/raw/main/titanic/test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd

datas = pd.read_csv("train.csv", encoding = "utf-8")
predicts = pd.read_csv("test.csv", encoding = "utf-8")


In [ ]:
predicts

In [ ]:
combine = pd.concat([datas, predicts], axis = 0, ignore_index = True) # 重設 預設ID
combine = combine.drop(["PassengerId", "Survived"], axis = 1) # 建立去除此兩欄的新表
combine

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1304,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
def change(c):
  if pd.isna(c):
    return c
  else:
    return c[0]
combine["Cabin"] = combine["Cabin"].apply(change)
combine["Cabin"]

0       NaN
1         C
2       NaN
3         C
4       NaN
       ... 
1304    NaN
1305      C
1306    NaN
1307    NaN
1308    NaN
Name: Cabin, Length: 1309, dtype: object

In [ ]:
# n = "Braund, Mr. Owen Harris"
# n.split(".")[0].split(",")[-1].strip() # strip 去掉前後空格
def change(n):
  if pd.isna(n):
    return n
  else:
    return n.split(".")[0].split(",")[-1].strip()
combine["Name"] = combine["Name"].apply(change)
combine["Name"]

0           Mr
1          Mrs
2         Miss
3          Mrs
4           Mr
         ...  
1304        Mr
1305      Dona
1306        Mr
1307        Mr
1308    Master
Name: Name, Length: 1309, dtype: object

In [ ]:
count = combine["Ticket"].value_counts() # Series 可當作 Dict 處理
def change(t):
  if pd.isna(t):
    return t
  else:
    return count[t]
# combine["Ticket"]
combine["Ticket"] = combine["Ticket"].apply(change)
combine["Ticket"]

0       1
1       2
2       1
3       2
4       1
       ..
1304    1
1305    3
1306    1
1307    1
1308    3
Name: Ticket, Length: 1309, dtype: int64

In [ ]:
combine
combine.isna() # 確認哪些是空值
s = combine.isna().sum() # 確認每欄有幾個空值

In [ ]:
" 留取資料 "
# Pandas 的篩選操作 Filter
# DataFrame / Series [帶入跟你資料筆數一樣多的True/False]
pd.DataFrame([
    [1, 2],
    [3, 4],
    [5, 6]
])[[True, False, True]] #True外層[] 為袋子, 裝著資料的容器, 容器內只留1,3筆資料
pd.Series([1, 2, 3])[[True, False, True]] 

s > 0 # 將每欄 > 0 呈現, 即為布林判斷
s[s > 0] # 如剛剛的留取資料概念
s[s > 0].sort_values(ascending = False) # 按價值排序, 不採升冪排列 (即以降冪排列)

Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

In [ ]:
# 不能有空的值 -> 應該補最有可能的值, 如下所示
# 類別值(固定數值) -> ex: Embarked, Pclass, Name(中間), Sex, Cabin
#  -> 最常出現的值
# 數值(無限多可能) -> ex: Age, SibSp, Parch, Ticket, Fare
#  -> 中間值(中位數)

In [ ]:
" 補類別值 -> 補最常出現的值 "
combine["Embarked"].value_counts() # 取得 Embarked 欄位內值的數量
combine["Embarked"].value_counts().max() # 取得 Embarked 類別內數量最多的數量
mostidx = combine["Embarked"].value_counts().idxmax() # 取得 Embarked 內最多數量的值

combine["Embarked"] = combine["Embarked"].fillna(mostidx) # fillna 空值欄填入資料, mostidx 為[(最多的類別)的值]

s = combine.isna().sum()
s[s > 0].sort_values(ascending = False) # 檢查資料缺失用

Cabin    1014
Age       263
Fare        1
dtype: int64

In [ ]:
" 補數值 -> 補中位數值 "
combine.median() # 取得 combine 內所有欄位數值的中位數, 但 Pclass 不是數值, 而是類別值, 捨掉為佳
med = combine.median().drop("Pclass")

combine = combine.fillna(med)

s = combine.isna().sum()
s[s > 0].sort_values(ascending = False) # 檢查資料缺失用

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


Cabin    1014
dtype: int64

In [56]:
""" 
資料處理方式
1.補缺失值
  1.1 類別值:補最多值 / One-Hot Encoding
  1.2 數值: 中位數
2.One-Hot encoding (類別值, 非數值比大小判斷, 資料夠多) -> 以決策樹做想像

"""
combine["Name"].value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Mlle              2
Major             2
Ms                2
Lady              1
Sir               1
Mme               1
Don               1
Capt              1
the Countess      1
Jonkheer          1
Dona              1
Name: Name, dtype: int64

In [ ]:
" 對類別做 One-Hot Encoding "
# 做 One-Hot Encoding 一定無礙, 但能否偷懶
# Pclass 有大小關係的類別, 補: 最常出現的值, One-Hot Encoding:可做可不做, 但試過才知道有無更佳效益
# Sex 僅二值, 可做可不做, male -> 0, female -> 1